In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
# !{sys.executable} -m pip install pandas==0.23.1

In [1]:
# !conda install -c conda-forge rdkit -y

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel

In [2]:
from pytorch_geometric2 import *

In [3]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import datetime
import random

# import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# from torch_geometric.data import Data, DataLoader
# import torch
# import torch.nn.functional as F
# from torch_geometric.datasets import Planetoid
# from torch_geometric.datasets import QM9
# from torch_geometric.datasets import TUDataset
# import torch_geometric.transforms as T
# from torch_geometric.nn import GCNConv, ChebConv  # noqa
# from torch.nn import Sequential, Linear, ReLU, GRU
# import torch_geometric.transforms as T
# from torch_geometric.datasets import QM9
# from torch_geometric.nn import NNConv, Set2Set
# from torch_geometric.data import DataLoader
# from torch_geometric.utils import remove_self_loops
# from torch_geometric.data import Data

from torch.utils.data import DataLoader

In [4]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

import numpy as np
from sklearn.decomposition import PCA

In [5]:
file_folder = '../../data/input'
os.listdir(file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [6]:
graph_dir='../../data/temp/pytorch_geometric2'

In [7]:
# run_convert_to_graph(graph_dir='../../data/temp/pytorch_geometric2', csv_dir=file_folder)

In [8]:
train = pd.read_csv(f'{file_folder}/train.csv')

In [9]:
test = pd.read_csv(f'{file_folder}/train.csv')

In [10]:
structures = pd.read_csv(f'{file_folder}/structures.csv')

In [11]:
molecule_names_test = test.molecule_name.unique().tolist()

In [12]:
molecule_names = train.molecule_name.unique().tolist()

In [13]:
train_dataset = ChampsDataset(molecule_names=molecule_names, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)
test_dataset = ChampsDataset(molecule_names=molecule_names_test, graph_file=graph_dir , csv='train', mode ='train', split=None, augment=None)



In [ ]:
list_ = []
df_node = pd.DataFrame()
for molecule_name, dataset in tqdm_notebook(zip(molecule_names, train_dataset), total=len(molecule_names)):
    d_ = {}
    for idx, row in train[train['molecule_name']==molecule_name].iterrows():
        edge_index_ = np.array([row['atom_index_0'], row['atom_index_1']])
        d_['id'] = row['id']
        for edge, edge_index in zip(dataset.__dict__['edge'], dataset.__dict__['edge_index']):
            if np.where(edge_index_==edge_index)[0].shape[0] == len(edge_index):
                for attribute_idx, edge_attribute in enumerate(edge):
                    d_[f'edge_{attribute_idx}'] = edge_attribute
                break
    pca = PCA(n_components=1)
    node_pca = pca.fit_transform(dataset.__dict__['node'])  
    df_node_i = structures[structures['molecule_name']==molecule_name][['molecule_name', 'atom_index']].copy()
    df_node_i['node_pca'] = node_pca[:,0]
    df_node = pd.concat([df_node, df_node_i], axis=0)
    list_.append(d_)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

